## Summary

- You should default to `[List[Object]]` which is close to `[object[]]` 
- unless you want a specific type, then use `[List[TypeName]]` which was `[TypeName[]]`

```ps1
[list[object]]$items = @()
[list[int]]$items = @()
```

You can ensure that even assigning the list to null: `$items = $null`
will not remove the list type by adding the `[ValidateNotNull()]` attribute. 
```ps1
# You can use attributes on variables? Oh yeah :)
[ValidateNotNull()]
[List[Int]]$alwaysNum = @()
```

Its best practice to use `List` generic types, verses the old, untyped `ArrayList`s.

[Some of the reasons why are described in the blue box here](https://learn.microsoft.com/en-us/dotnet/api/System.Collections.ArrayList?view=net-6.0) and [in the docs for `List`](https://learn.microsoft.com/en-us/dotnet/api/system.collections.generic.list-1?view=net-6.0#remarks)

- `[List[type]]` is typesafe (its strongly typed)
- You can implement  `IComparable` and `IEquateable` ( verses `ArrayList` which is always type `object` -- so you have to add additional code)
- `ArrayList` is untyped, it's a list of objects. It's from before generics existed.
- you can tie into automatic data validation from attributes or `class-records` . Great for validating JSON for web apis (see other notebook)

It's super easy to update:

In [1]:
using namespace System.Collections.Generic
# replace these 
$arrayList = new-object system.collections.arraylist 
$arrayList = [ArrayList]::new()

# with generics
# or if you want to be strict, you can use a single type
$items = [list[object]]::new()
$items = [list[int]]::new()

# The constructors above are essentially the same as these
[list[object]]$items = @()
[list[int]]$items = @()

# adding elements, this
$arrayList.Add( $stuff )

# doesn't change
$items.add( $stuff )


InvalidOperation: 
Line |
   3 |  $arrayList = [ArrayList]::new()
     |               ~~~~~~~~~~~
     | Unable to find type [ArrayList].
InvalidOperation: 
Line |
   7 |  $items = [list[object]]::new()
     |           ~~~~~~~~~~~~~~
     | Unable to find type [list].
InvalidOperation: 
Line |
   8 |  $items = [list[int]]::new()
     |           ~~~~~~~~~~~
     | Unable to find type [list].
InvalidOperation: 
Line |
  11 |  [list[object]]$items = @()
     |  ~~~~~~~~~~~~~~
     | Unable to find type [list].
InvalidOperation: 
Line |
  12 |  [list[int]]$items = @()
     |  ~~~~~~~~~~~
     | Unable to find type [list].
0
InvalidOperation: 
Line |
  18 |  $items.add( $stuff )
     |  ~~~~~~~~~~~~~~~~~~~~
     | You cannot call a method on a null-valued expression.


In [2]:
"You can also use implicit arrays created by the pipeline
.Add() isn't required"

$numbers            = 0..100 | %{ $_ * 3 }
[List[Int]]$numbers = 0..100 | %{ $_ * 3 }


You can also use implicit arrays created by the pipeline
.Add() isn't required
InvalidOperation: 
Line |
   5 |  [List[Int]]$numbers = 0..100 | %{ $_ * 3 }
     |  ~~~~~~~~~~~
     | Unable to find type [List].


### Strongly Typing your List

You may ask: Isn't this example already strongly typed?
The answer is yes, but, technically no

```ps1
$letters = [List[String]]::new()

# the strongly typed version:
[list[string]]$letters = @()
```

`$letters` is assigned to the strongly typed list, but, `$letters` itself is untyped. Normally you won't see much of a difference.

### the Short Version

```ps1

$numbers = [List[Int]]::new() # is: [List[Int]]
$numbers.Add( 42.6 )   # is: [List[Int]], 42.6 is  coerced to int
$numbers.GetType()     # is: [List[Int]]
$numbers.Add( 'abcd' ) # is: [List[Int]]

'Exception: Cannot convert argument "item", with value: "abcd", for "Add" to type "System.Int32"'

$numbers = 'abcd'      # is: [String]
$numbers.Add( 42.6 )   # is: [String]
"Exception: Method invocation failed because [System.String] does not contain a method named 'Add'"
```
Now with strong typing
```ps1
[List[Int]]$alwaysNum = @() # is: [List[Int]]
$alwaysNum.Add( 42.6 )      # is: [List[Int]]
$alwaysNum.Add( 'abcd' )    # is: [List[Int]]
'Exception: Cannot convert argument "item", with value: "abcd", for "Add" to type "System.Int32"'

$alwaysNum = 'abcd'         # is: [List[Int]]
'Exception: Cannot convert value "abcd" to type "System.Int32"'
```

### The longer version

In [3]:

$numbers = [list[int]]::new()
$numbers.Add('234')
$numbers.GetType().tostring() # Is: [List<int>]

$numbers.Add( 'asdf' )

# MethodException: Cannot convert argument "item", with value: "abcd", for "Add" to type "System.Int32": "Cannot convert value "abcd" to type "System.Int32". Error: "Input string was not in a correct format.""

"It threw an exception, so it's looking good?"

InvalidOperation: 
Line |
   2 |  $numbers = [list[int]]::new()
     |             ~~~~~~~~~~~
     | Unable to find type [list].
MethodInvocationException: 
Line |
   3 |  $numbers.Add('234')
     |  ~~~~~~~~~~~~~~~~~~~
     | Exception calling "Add" with "1" argument(s): "Collection was of a fixed size."
System.Object[]
MethodInvocationException: 
Line |
   6 |  $numbers.Add( 'asdf' )
     |  ~~~~~~~~~~~~~~~~~~~~~~
     | Exception calling "Add" with "1" argument(s): "Collection was of a fixed size."
It threw an exception, so it's looking good?


In [4]:
$numbers = 'abcd' 
$numbers.add(434) # is: [string]  

# MethodException: Cannot convert argument "item", with value: "abcd", for "Add" to type "System.Int32": "Cannot convert value "abcd" to type "System.Int32". Error: "Input string was not in a correct format.""

'The datatype of numbers is now a string, it changed'


InvalidOperation: 
Line |
   2 |  $numbers.add(434) # is: [string]
     |  ~~~~~~~~~~~~~~~~~
     | Method invocation failed because [System.String] does not contain a method named 'Add'.
The datatype of numbers is now a string, it changed


In [5]:

[list[int]]$alwaysInt = @()
$alwaysInt.Add(42.3)        # type: [list[Int]]
$alwaysInt.Add('abcd')      # type: [list[Int]]
'It coerced 42.3 to an int, but could not convert text to an int,
 on adding good Good so far'

InvalidOperation: 
Line |
   2 |  [list[int]]$alwaysInt = @()
     |  ~~~~~~~~~~~
     | Unable to find type [list].
InvalidOperation: 
Line |
   3 |  $alwaysInt.Add(42.3)        # type: [list[Int]]
     |  ~~~~~~~~~~~~~~~~~~~~
     | You cannot call a method on a null-valued expression.
InvalidOperation: 
Line |
   4 |  $alwaysInt.Add('abcd')      # type: [list[Int]]
     |  ~~~~~~~~~~~~~~~~~~~~~~
     | You cannot call a method on a null-valued expression.
It coerced 42.3 to an int, but could not convert text to an int,
 on adding good Good so far


In [6]:
$alwaysInt.GetType().ToString()
$alwaysInt = 'abcd'               # type: [List[Int]]
$alwaysInt.GetType().ToString()
'Not only did it not assign the value to a string,
it also preserved its original type'

InvalidOperation: You cannot call a method on a null-valued expression.
System.String
Not only did it not assign the value to a string,
it also preserved its original type


In [7]:

$numbers = [List[Int]]::new()
[List[Int]]$strongNumbers = @()


$numbers.GetType() # is: [list[object]]
$numbers = '123'
$numbers.GetType() # is: [string]

$strongNumbers.GetType() # is: [list[object]]
$strongNumbers = '123'
$strongNumbers.GetType() # is: [list[object]]

$strongNumbers.GetType() # is: [list[object]]
$strongNumbers = '123'
$strongNumbers.GetType() # is: [list[object]]




#$compliantList = [Collections.Generic.List[object]]::new()

InvalidOperation: 
Line |
   2 |  $numbers = [List[Int]]::new()
     |             ~~~~~~~~~~~
     | Unable to find type [List].
InvalidOperation: 
Line |
   3 |  [List[Int]]$strongNumbers = @()
     |  ~~~~~~~~~~~
     | Unable to find type [List].

IsPublic IsSerial Name                                     BaseType
-------- -------- ----                                     --------
True     True     String                                   System.Object
True     True     String                                   System.Object
InvalidOperation: 
Line |
  10 |  $strongNumbers.GetType() # is: [list[object]]
     |  ~~~~~~~~~~~~~~~~~~~~~~~~
     | You cannot call a method on a null-valued expression.
True     True     String                                   System.Object
True     True     String                                   System.Object
True     True     String                                   System.Object

